# Intro
In this notebook, we showcase how to fine-tune the Qwen3-1.7B model on AWS Trainium using the Hugging Face Optimum Neuron library.
The goal of this task is Text-to-SQL generation — training the model to translate natural language questions into executable SQL queries.

We will fine-tune the model using `optimum.neuron`, save the trained checkpoint, and then deploy it for inference with Optimum-Neuron[vllm], enabling high-performance, low-latency Text-to-SQL execution.

By the end of this notebook, you’ll have a fine-tuned, Trainium-optimized Qwen3 model ready for deployment and real-time inference. This workflow demonstrates how to leverage the Optimum Neuron toolchain to efficiently train and serve large language models on AWS Neuron devices.

For this module, you will be using the [b-mc2/sql-create-context](https://huggingface.co/datasets/b-mc2/sql-create-context) dataset which consists of thousands of examples of SQL schemas, questions about the schemas, and SQL queries intended to answer the questions.

*Dataset example 1:*
* *SQL schema/context:* `CREATE TABLE management (department_id VARCHAR); CREATE TABLE department (department_id VARCHAR)`
* *Question:* `How many departments are led by heads who are not mentioned?`
* *SQL query/answer:* `SELECT COUNT(*) FROM department WHERE NOT department_id IN (SELECT department_id FROM management)`

*Dataset example 2:*
* *SQL schema/context:* `CREATE TABLE courses (course_name VARCHAR, course_id VARCHAR); CREATE TABLE student_course_registrations (student_id VARCHAR, course_id VARCHAR)`
* *Question:* `What are the ids of all students for courses and what are the names of those courses?`
* *SQL query/answer:* `SELECT T1.student_id, T2.course_name FROM student_course_registrations AS T1 JOIN courses AS T2 ON T1.course_id = T2.course_id`

By fine-tuning the model over several thousand of these text-to-SQL examples, the model will then learn how to generate an appropriate SQL query when presented with a SQL context and a free-form question.

This text-to-SQL use case was selected so you can successfully fine-tune your model in a reasonably short amount of time (~25 minutes) which is appropriate for this workshop. Although this is a relatively simple use case, please keep in mind that the same techniques and components used in this module can also be applied to fine-tune LLMs for more advanced use cases such as writing code, summarizing documents, creating blog posts - the possibilities are endless!

# Install requirements
This notebook uses [Hugging Face Optimum Neuron](https://github.com/huggingface/optimum-neuron) which works like an interface between the Hugging Face Transformers library and AWS Accelerators including AWS Trainium and AWS Inferentia. We will also install some other libraries like peft, trl etc.


In [1]:
%cd /home/ubuntu/environment/FineTuning/HuggingFaceExample/01_finetuning/assets
%pip install -r requirements.txt

/home/ubuntu/environment/FineTuning/HuggingFaceExample/01_finetuning/assets


/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Fine-tuning

In this section, we fine-tune the Qwen3-1.7B model on the Text-to-SQL task using Hugging Face Optimum Neuron. Here are the parameters we are going to pass - 

1. `--nnodes`:	Number of nodes (1 = single node)
2. `--nproc_per_node`: 	Processes per node (usually equals number of devices).
3. `--model_id, --tokenizer_id`:	Model and tokenizer identifiers (from Hugging Face or local path).
4. `--output_dir`:	Directory for saving checkpoints and logs.
5. `--bf16`:	Enables bfloat16 precision for faster, memory-efficient training.
5. `--gradient_checkpointing`:	Saves memory by recomputing activations during backprop.
6. `--gradient_accumulation_steps`:	Steps to accumulate gradients before optimizer update.
7. `--learning_rate`:	Initial training learning rate.
8. `--max_steps`:	Total number of training steps.
9. `--per_device_train_batch_size`:	Batch size per device.
10. `--tensor_parallel_size`:	Number of devices for tensor parallelism.
11. `--lora_r, --lora_alpha, --lora_dropout`:	LoRA hyperparameters — rank, scaling, and dropout rate.
12. `--dataloader_drop_last`:	Drops last incomplete batch.
13. `--disable_tqdm`: Disables progress bar.
14. `--logging_steps`:	Log interval (in steps).

In [8]:
!python -c "import neuronx_distributed as nxd; print('neuronx_distributed imported OK'); print([a for a in dir(nxd) if 'initialize' in a or 'parallel' in a][:20])"

/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/neuronx_distributed/parallel_layers/layers.py:14: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from .mappings import (
/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/neuronx_distributed/parallel_layers/layers.py:14: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from .mappings import (
/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/neuronx_distributed/parallel_layers/layers.py:14: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from .mappings import (
/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/neuronx_distributed/modules/moe/blockwise.py:42: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  component, error = import_nki(config)
/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/neuronx_distributed/modules/moe/blockwise

In [ ]:
!torchrun \
  --nnodes 1 \
  --nproc_per_node 2 \
  finetune_model.py \
  --model_id Qwen/Qwen3-1.7B \
  --tokenizer_id Qwen/Qwen3-1.7B \
  --output_dir ~/environment/ml/qwen \
  --bf16 True \
  --gradient_checkpointing True \
  --gradient_accumulation_steps 1 \
  --learning_rate 5e-5 \
  --max_steps 1000 \
  --per_device_train_batch_size 2 \
  --tensor_parallel_size 2 \
  --lora_r 16 \
  --lora_alpha 32 \
  --lora_dropout 0.05 \
  --dataloader_drop_last True \
  --disable_tqdm True \
  --logging_steps 10

W1108 22:28:27.825000 3332 torch/distributed/run.py:766] 
W1108 22:28:27.825000 3332 torch/distributed/run.py:766] *****************************************
W1108 22:28:27.825000 3332 torch/distributed/run.py:766] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W1108 22:28:27.825000 3332 torch/distributed/run.py:766] *****************************************
/opt/aws_neuronx_venv_pytorch_2_7_nxd_inference/lib/python3.10/site-packages/neuronx_distributed/parallel_layers/layers.py:14: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from .mappings import (
/opt/aws_neuronx_venv_pytorch_2_7_nxd_inference/lib/python3.10/site-packages/neuronx_distributed/parallel_layers/layers.py:14: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from .mappings import (
/opt/aws_neuron

# Compilation

After completing the fine-tuning process, the next step is to compile the trained model for AWS Trainium inference using the Hugging Face Optimum Neuron toolchain.
Neuron compilation optimizes the model graph and converts it into a Neuron Executable File Format (NEFF), enabling efficient execution on NeuronCores.

In [4]:
!optimum-cli export neuron \
  --model /home/ubuntu/environment/ml/qwen/merged_model \
  --task text-generation \
  --sequence_length 512 \
  --batch_size 1 \
  /home/ubuntu/environment/ml/qwen/compiled_model

/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/optimum/commands/env.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution
/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/neuronx_distributed/parallel_layers/layers.py:14: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from .mappings import (
/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/neuronx_distributed/parallel_layers/layers.py:14: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from .mappings import (
/opt/aws_neuronx_venv_pytorch_latest/lib/python3.10/site-packages/neuronx_distributed/parallel_layers/layers.py:14: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nk

# Inference

We will install the Optimum Neuron vllm library.  Then, run inference using the compiled model.

In [5]:
%pip install optimum-neuron[vllm]


Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached torchvision-0.22.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (6.1 kB)
  Using cached astor-0.8.1-py2.py3-none-any.whl.metadata (4.2 kB)
INFO: pip is looking at multiple versions of opencv-python-headless to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 MB 77.2 MB/s  0:00:03m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 35.9 MB/s  0:00:10m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 168.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 175.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.4/156.4 MB 144.8 MB/s  0:00:010:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 156.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 171.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [13]:
import os
from vllm import LLM, SamplingParams
llm = LLM(
    model="/home/ubuntu/environment/ml/qwen/compiled_model", #local compiled model
    max_num_seqs=1,
    max_model_len=2048,
    device="neuron",
    tensor_parallel_size=2,
    override_neuron_config={})
example1="""
<|im_start|>system
You are medical diagnoser. 
The patient will talk to you about their medical condition or illnesses of symptoms they might have.
You have to advise them appropriately. 
Mention any relevant diseases, health checks and medicines that the patient should look into.<|im_end|>
<|im_start|>user
I got 104 fever, what medicine do i take? <|im_end|>
<|im_start|>assistant
"""

prompts = [
    example1
]

sampling_params = SamplingParams(max_tokens=2048, temperature=0.8)
outputs = llm.generate(prompts, sampling_params)

print("#########################################################")

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, \n\n Generated text: {generated_text!r} \n")

INFO 11-08 23:00:06 [config.py:841] This model supports multiple tasks: {'reward', 'generate', 'embed', 'classify'}. Defaulting to 'generate'.
INFO 11-08 23:00:06 [config.py:1472] Using max model len 2048
INFO 11-08 23:00:06 [llm_engine.py:230] Initializing a V0 LLM engine (v0.9.2) with config: model='/home/ubuntu/environment/ml/qwen/compiled_model', speculative_config=None, tokenizer='/home/ubuntu/environment/ml/qwen/compiled_model', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=True, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cpu, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_backend=''), observability_config

INFO:Neuron:Loading sharded checkpoint from /home/ubuntu/environment/ml/qwen/compiled_model/checkpoint/weights


WARNING 11-08 23:00:07 [config.py:4861] Current vLLM config is not set.
INFO 11-08 23:00:07 [executor_base.py:113] # neuron blocks: 2, # CPU blocks: 0
INFO 11-08 23:00:07 [executor_base.py:118] Maximum concurrency for 2048 tokens per request: 2.00x
INFO 11-08 23:00:07 [llm_engine.py:428] init engine (profile, create kv cache, warmup model) took 0.00 seconds


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

#########################################################
Prompt: '\n<|im_start|>system\nYou are medical diagnoser. \nThe patient will talk to you about their medical condition or illnesses of symptoms they might have.\nYou have to advise them appropriately. \nMention any relevant diseases, health checks and medicines that the patient should look into.<|im_end|>\n<|im_start|>user\nI got 104 fever, what medicine do i take? <|im_end|>\n<|im_start|>assistant\n', 

 Generated text: '<think>\n\n</think>\n\nhello I understand your concern take fever and pain relief medicine acetaminophen paracetamol twice daily and also take antipyretic tablet along with it for 10 days consult a doctor and get her examined and investigated further for the etiology of fever and also get the blood sugar level tested and also get your urine examined and urine culture and sensitivity sensitive antibacterial medicines may be started after blood examination and urine culture and sensitivity test if there is any ur

In [14]:
!cd /home/ubuntu/environment/ml/qwen

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
!ls

consolidate_adapter_shards_and_merge_model.py  finetune_model_copy_2.py
finetune_model.py			       requirements.txt
finetune_model_copy.py


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
